In [1]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=10)

from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy

candidate_set_size = 1000 if not SMOKE_TEST else 5
candidate_set = torch.rand(
    candidate_set_size, bounds_norm.size(1), device=bounds.device, dtype=bounds.dtype
)
qGIBBON = qLowerBoundMaxValueEntropy(model, candidate_set)
scipy.__version__

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


CPU times: user 2.83 s, sys: 606 ms, total: 3.43 s
Wall time: 4.16 s


'1.14.1'

In [2]:
%%time

# prep different acqusition functions
candidate_set = torch.rand(
    10000, bounds.size(1), device=bounds.device, dtype=bounds.dtype
)
acqs = {}
acqs["aa"] = qLowerBoundMaxValueEntropy(model, candidate_set)
acqs["MES"] = qMaxValueEntropy(model, candidate_set)
acqs["EI"] = qNoisyExpectedImprovement(model, train_X)
acqs["PI"] = qProbabilityOfImprovement(model, best_f=train_Y.max())

# prep grid to evaluate acq functions
n = 100 if not SMOKE_TEST else 2
xv, yv = torch.meshgrid([torch.linspace(0, 1, n), torch.linspace(0, 1, n)])
test_x = torch.stack([xv.reshape(n * n, 1), yv.reshape(n * n, 1)], -1)

# eval and maximise acq functions

with cProfile.Profile() as pr:
    for acq in acqs.values():
        acq(test_x).detach().reshape(n, n)
        optimize_acqf(
            acq_function=acq,
            bounds=bounds_norm,
            q=3,
            num_restarts=5,
            raw_samples=100,
            sequential=True,
        )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()

scipy.__version__

/Users/saitcakmak/botorch/botorch/acquisition/monte_carlo.py:496: NumericsWarning: qNoisyExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qNoisyExpectedImprovement 	 --> 	 qLogNoisyExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
/opt/anaconda3/envs/botorch/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


         2674263 function calls (2566248 primitive calls) in 2.449 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    2.435    0.609 /Users/saitcakmak/botorch/botorch/optim/optimize.py:479(optimize_acqf)
        4    0.000    0.000    2.435    0.609 /Users/saitcakmak/botorch/botorch/optim/optimize.py:630(_optimize_acqf)
        4    0.000    0.000    2.435    0.609 /Users/saitcakmak/botorch/botorch/optim/optimize.py:244(_optimize_acqf_sequential_q)
       12    0.000    0.000    1.530    0.127 /Users/saitcakmak/botorch/botorch/optim/optimize.py:310(_optimize_acqf_batch)
       12    0.000    0.000    1.433    0.119 /Users/saitcakmak/botorch/botorch/optim/optimize.py:353(_optimize_batch_candidates)
       12    0.000    0.000    1.432    0.119 /Users/saitcakmak/botorch/botorch/generation/gen.py:44(gen_candidates_scipy)
       12    0.000    0.000    1.401    0.117 /Users/saitcakmak/botorch/botor

'1.14.1'